In [2]:
from typing import Tuple

import torch
from torch.utils.data import DataLoader, Dataset

from config import *

from prediction_models.dbn import *
from prediction_models.kelm import *

%reload_ext autoreload
%autoreload 2

In [3]:
DBN_LAYER_SIZES = CONFIG['DBN_LAYER_SIZES']
INPUT_SIZE = DBN_LAYER_SIZES[0]
HIDDEN_LAYER_SIZES = DBN_LAYER_SIZES[1:]
GIBBS_SAMPLING_STEPS = CONFIG['GIBBS_SAMPLING_STEPS']

### Pre-train RBMs inside DBN

Train to reconstruct all possible combinations of 0's and 1's

#### Create a dataset

In [4]:
class BinaryVectorDataset(Dataset):
    def __init__(self, n_bits: int):
        self.n_bits = n_bits
        self.format_str = f'{{0:0{self.n_bits}b}}'
    
    def __len__(self) -> int:
        return 2 ** self.n_bits
    
    def __getitem__(self, index: int) \
            -> Tuple[torch.TensorType, torch.TensorType]:
        bin_str = self.format_str.format(index)
        return torch.tensor([float(c) for c in bin_str]), torch.tensor([index,])

dbn_pre_train_loader = DataLoader(BinaryVectorDataset(INPUT_SIZE))

#### Create and pre-train DBN

In [5]:
dbn = DBN(INPUT_SIZE, HIDDEN_LAYER_SIZES, k=GIBBS_SAMPLING_STEPS)
pre_train_dbm(dbn, dbn_pre_train_loader)

/home/kolusask/miniconda3/envs/.idp/lib/python3.11/site-packages/torch/autograd/__init__.py:200: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 0, Machine 0:	Loss: -1.5616679191589355
Epoch 0, Machine 1:	Loss: -0.27533671259880066
Epoch 0, Machine 2:	Loss: -0.5047240257263184
Epoch 10, Machine 0:	Loss: -2.8526511192321777
Epoch 10, Machine 1:	Loss: -0.4449526369571686
Epoch 10, Machine 2:	Loss: -0.874451756477356


DBN(
  (rbms): ModuleList(
    (0-2): 3 x RBM()
  )
)